# EM LATAM Credit 5Y CDS Basis Trading Strategy

In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# -------------------- Fetch 5Y UST Gov Bond Yield and 5Y CDS Spreads of LATAM Countries --------------------

us_5y_gov = pd.read_csv('/Volumes/SanDisk - External SSD/EM Credit/us_5yr_govt.csv')
mx_5y_cds = pd.read_csv('/Volumes/SanDisk - External SSD/EM Credit/mexico_5y_cds.csv')
br_5y_cds = pd.read_csv('/Volumes/SanDisk - External SSD/EM Credit/brazil_5yr_cds.csv')
ar_5y_cds = pd.read_csv('/Volumes/SanDisk - External SSD/EM Credit/argentina_5y_cds.csv')
cl_5y_cds = pd.read_csv('/Volumes/SanDisk - External SSD/EM Credit/chile_5y_cds.csv')

In [3]:
cl_5y_cds

,Date,PX_LAST
0,31/03/2023,104.242
1,03/04/2023,103.754
2,04/04/2023,105.553
3,05/04/2023,109.822
4,06/04/2023,110.392
...,...,...
518,26/03/2025,59.320
519,27/03/2025,59.113
520,28/03/2025,60.461
521,31/03/2025,60.342


In [2]:
# -------------------- Trading Strategy with Bloomberg Data --------------------
# Define data for countries (with Bloomberg data)
data = {
    "Risk Free Rate": us_5y_gov["PX_LAST",]
    "currency": ["USD", "USD", "USD", "USD"],  # All USD-denominated
}

df = pd.DataFrame(data)

# Update bond yields and CDS spreads with Bloomberg data
df["bond_yield"] = bloomberg_df["USGG10YR Index"]  # Example bond yields
df["cds_spread"] = bloomberg_df["MXGB10YR Index"]  # Example CDS spreads for Mexico (adjust as needed)

# Risk-Free Rate Selection based on currency
df["risk_free_rate"] = df["currency"].apply(lambda x: risk_free_rates["5Y"] if x == "USD" else np.nan)

# Example of local government bond yields for non-USD countries
local_gov_bond_yields = {"Mexico": 0.08, "Brazil": 0.095, "Chile": 0.06}  # Exclude Argentina as it is USD
df["risk_free_rate"] = df.apply(lambda row: local_gov_bond_yields[row["issuer"]] if row["currency"] != "USD" else row["risk_free_rate"], axis=1)

# Z-spread Calculation and CDS Basis
df["z_spread"] = df["bond_yield"] - df["risk_free_rate"]
df["cds_basis"] = df["z_spread"] - df["cds_spread"]

# -------------------- Repo Rates --------------------
repo_rates = {"Mexico": 0.02, "Brazil": 0.015, "Chile": 0.02, "Argentina": 0.03}
df["repo_rate"] = df["issuer"].map(repo_rates)

# Adjust for repo rates and bid-ask spread
df["bid_ask_spread"] = 0.002
df["adjusted_basis"] = df["cds_basis"] - df["repo_rate"] - df["bid_ask_spread"]

# -------------------- Machine Learning for Trade Signals --------------------
df["volatility"] = np.random.uniform(0.01, 0.05, len(df))
df["credit_slope"] = df["cds_spread"].diff().fillna(0)
df["profitable_trade"] = np.where(df["adjusted_basis"].abs() > 0.005, 1, 0)

# Features and target for Random Forest model
X = df[["adjusted_basis", "volatility", "credit_slope"]]
y = df["profitable_trade"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
df["trade_signal"] = model.predict(X)

# -------------------- CVaR Calculation --------------------
def simulate_cds_basis(mu, sigma, n_simulations=10000, days=5):
    dt = 1 / 252
    simulated_basis = np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.randn(n_simulations, days))
    return simulated_basis[:, -1]

def compute_cvar(simulated_losses, confidence_level=0.95):
    var_threshold = np.percentile(simulated_losses, (1 - confidence_level) * 100)
    cvar = simulated_losses[simulated_losses <= var_threshold].mean()
    return var_threshold, cvar

# Simulate Losses
simulated_losses = simulate_cds_basis(mu=-0.012, sigma=0.025)
var_95, cvar_95 = compute_cvar(simulated_losses)
df["VaR_95"] = df["volatility"] * norm.ppf(0.95) * np.sqrt(5)

# -------------------- Regime Switching Model --------------------
def detect_market_regime(cds_spreads):
    hmm_model = GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    hmm_model.fit(cds_spreads.reshape(-1, 1))
    hidden_states = hmm_model.predict(cds_spreads.reshape(-1, 1))
    return hidden_states, hmm_model

# Example CDS Spreads (use actual data in practice)
np.random.seed(42)
cds_spreads = np.cumsum(np.random.randn(100) * 0.005) + 0.05
market_regime, hmm_model = detect_market_regime(cds_spreads)

# Add regime information to DataFrame
df["Regime"] = market_regime[-len(df):]

# -------------------- Dynamic Position Sizing --------------------
def dynamic_position_sizing(regime, cvar, base_position=1_000_000):
    if regime == 0:  # Risk-On
        return base_position * 1.5
    elif regime == 1:  # Risk-Off
        return base_position * 0.5
    else:
        return base_position

# Apply position sizing logic
df["Position Size"] = df["Regime"].apply(lambda r: dynamic_position_sizing(r, cvar_95))

# Final Trade Signal (Trade only if profitable signal and within VaR limits)
df["Trade Final"] = np.where((df["trade_signal"] == 1) & (df["VaR_95"] < 0.02), "Trade", "No Trade")

# -------------------- Visualizations --------------------
# 1. CDS Basis and Bond Yields over Time
plt.figure(figsize=(10, 6))
plt.plot(df["bond_yield"], label='Bond Yield (10Y)', color='blue')
plt.plot(df["cds_basis"], label='CDS Basis', color='red')
plt.title('Bond Yields and CDS Basis over Time')
plt.xlabel('Time')
plt.ylabel('Yield / CDS Basis')
plt.legend()
plt.show()

# 2. Adjusted CDS Basis and Repo Rates
plt.figure(figsize=(10, 6))
plt.plot(df["adjusted_basis"], label='Adjusted CDS Basis', color='green')
plt.plot(df["repo_rate"], label='Repo Rate', color='orange')
plt.title('Adjusted CDS Basis and Repo Rate')
plt.xlabel('Time')
plt.ylabel('Basis / Repo Rate')
plt.legend()
plt.show()

# 3. Regimes and Trade Signals
plt.figure(figsize=(10, 6))
plt.plot(df["Regime"], label='Market Regime (0=Risk-On, 1=Risk-Off)', color='purple')
plt.scatter(df.index, df["trade_signal"], label='Trade Signal', color='red', marker='x')
plt.title('Market Regimes and Trade Signals')
plt.xlabel('Time')
plt.ylabel('Regime and Signal')
plt.legend()
plt.show()

Failed to get ticker '^IRX' reason: Expecting value: line 1 column 1 (char 0)
^IRX: No price data found, symbol may be delisted (period=1d)
Failed to get ticker '^FVX' reason: Expecting value: line 1 column 1 (char 0)
^FVX: No price data found, symbol may be delisted (period=1d)
Failed to get ticker '^TNX' reason: Expecting value: line 1 column 1 (char 0)
^TNX: No price data found, symbol may be delisted (period=1d)


⚠ Warning: No data for ^IRX, using fallback rate.
⚠ Warning: No data for ^FVX, using fallback rate.
⚠ Warning: No data for ^TNX, using fallback rate.
         issuer currency  adjusted_basis  trade_signal    VaR_95 Trade Final  \
0        Brazil      USD          -0.027             1  0.057909    No Trade   
1        Mexico      USD          -0.027             1  0.128760    No Trade   
2  South Africa      ZAR          -0.122             1  0.127987    No Trade   

   Position Size  
0      1500000.0  
1      1500000.0  
2      1500000.0  


In [ ]:
main()